In [1]:
# https://stackoverflow.com/questions/21971449/how-do-i-increase-the-cell-width-of-the-jupyter-ipython-notebook-in-my-browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

C:\Users\pulzara\AppData\Local\Temp\ipykernel_14180\3814626351.py:2: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


# Librerias

In [3]:
import numpy as np
import pandas as pd
import random
import seaborn as sns 
import matplotlib.pyplot as plt
import plotly.express as px
import datetime
from datetime import datetime
from sklearn.metrics import mean_squared_error
from math import sqrt
import statsmodels
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from math import sqrt


import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger('fbprophet').setLevel(logging.WARNING)


import gc

# Carga de datos

In [4]:
data = pd.read_table('../data/Modelar_UH2022.txt', delimiter = '|', encoding = "UTF-8")

# Transformaciones

## Funciones

*La funcion ***ID*** devuelve un dataframe con los valores correspondientes de un contador especifico.* 

* *Se organizan de manera ascendente por una columna especifica (SAMPLETIME)*

In [19]:
def id(data, i, sort_value_column):
    """
    Esta funcion obtiene un numero de id de contador en especifico.
    Realiza un sort values
    """
    data = data[data.ID == i]
    data = data.sort_values(by= sort_value_column, ascending=True)
    data1 = data.reset_index(drop = True)  

    return data1 


La funcion ***hour*** cambia el formato de las fechas, pasando segundos y minutos a el valor cero. <br>
La salida de la columna viene dada por: 
* year-month-day hour:00:00 

In [20]:
def hour(data1, column_name):
    """
    cambia la fecha y lo deja en una hora sin minutos y segundos. 
    data, "column_name"
    """
    data1[column_name] = data1[column_name].str[:-5]
    data1[column_name] = data1[column_name] + '00:00'

    return data1 


La funcion ***negativo***  reemplaza los valores negativos de la columna DELTAINTEGER Y DELTATHOUSANDTH por cero. <br>
* No deben aparecer valores negativos en un contador.

In [21]:
def negativo(df):

    df["DELTAINTEGER"].mask(df["DELTAINTEGER"] < 0, 0, inplace = True)
    df["DELTATHOUSANDTH"].mask(df["DELTATHOUSANDTH"] < 0, 0, inplace = True)
    return "True"

La funcion ***deltainteger*** realiza varias operaciones con el fin de obtener la parte entera y decimal de los valores de consumo de agua en una sola columna. 
* Pasa la variable SAMPLETIME a un pandas Datetime
* Reemplaza los valores NaN por cero de todo el dataset
* Se cambia el formato de DELTATHOUSANDTH a tipo int despues a tipo str 
* Se cambia el formato de DELTAINTEGER a tipo str para ser una union con DELTATHOUSANDTH
* Se crea una nueva columna que se llama deltainteger
* Es necesario establecer deltainteger como una variable tipo float
* Elimina las columnas que no se necesitan y se queda con el ID, SAMPLETIME y deltainteger

In [22]:
def deltainteger(df): 

    """
    Esta funcion crea una nueva variable deltainteger que es la suma de DELTAINTEGER + DELTATHOUSAND
    Cambia a datetime la columna SAMPLETIME
    """

    df["SAMPLETIME"] = pd.to_datetime(df["SAMPLETIME"])
    df = df.replace(np.nan,0) 
    df['DELTATHOUSANDTH'] = df['DELTATHOUSANDTH'].astype(float).astype(int)
    df.sort_values(by="SAMPLETIME", inplace=True)
    df.reset_index(inplace=True, drop=True)
    df['DELTAINTEGER'] = df['DELTAINTEGER'].astype(str)
    df['DELTATHOUSANDTH'] = df['DELTATHOUSANDTH'].astype(str)
    df['deltainteger'] = df['DELTAINTEGER'] + '.' + df['DELTATHOUSANDTH'] 
    df['deltainteger'] = df['deltainteger'].astype(float)  
    df.drop(df.columns[[2,3,4,5]], axis = 1, inplace =True) 

    return df

La funcion ***group*** agrupa el dataset por frecuencia diaria, semanal o mensual. <br>
Esta acción se realiza por dos variables:

* SAMPLETIME
* ID

In [23]:
def group(data, column_name, frec):

    """
    Esta funcion agrupa por serie de tiempo en dias para obtener la media de deltainteger
    """
    data = data.groupby([pd.Grouper(key=column_name, freq=frec), "ID"]).sum()
    data = data.reset_index()
    data["SAMPLETIME"] = pd.to_datetime(data["SAMPLETIME"]) 
    data["SAMPLETIME"] = data["SAMPLETIME"].sort_values(ascending=False)
    data = data.set_index(column_name)

    return data 

Esta funcion en conjunto permite detectar la cantidad de outliers que se encuentran dentro de una columna del dataframe 
* Basado en los percentiles 0.25 y 0.75. La funcion ***remove_outliers*** elimina los valores por fuera del rango intercuartilico
* La funcion ***replace_outliers*** reemplaza los valores que se encuentran fuera del quantile 0.25 y 0.95 por la media
* La funcion outliers usa las funciones anteriores, detectanto la cantidad de outliers en la columna deltainteger. Si la cantidad de outliers esta por debajo del 5% los elimina, sino se queda con los datos originales. Funciona para IDs que no esten vacios.

In [31]:
def valores_iqr(df_in, col_name):
        median = df_in[col_name].median()
        q1 = df_in[col_name].quantile(0.25) # 25th percentile / 1st quartile
        q3 = df_in[col_name].quantile(0.75) # 7th percentile / 3rd quartile
        iqr = q3-q1 #Interquartile range
        minimum  = q1-1.5*iqr # The minimum value or the |- marker in the box plot
        maximum = q3+1.5*iqr # The maximum value or the -| marker in the box plot
        return median, q1, q3, iqr, minimum, maximum

def texto_iqr(df_in, col_name):
    median, q1, q3, iqr, minimum, maximum = valores_iqr(df_in, col_name)
    text = f"median={median:.2f}, q1={q1:.2f}, q3={q3:.2f}, iqr={iqr:.2f}, minimum={minimum:.2f}, maximum={maximum:.2f}"
    return text

def remove_outliers(df_in, col_name):
    _, _, _, _, minimum, maximum = valores_iqr(df_in, col_name)
    df_out = df_in.loc[(df_in[col_name] > minimum) & (df_in[col_name] < maximum)]
    return df_out

def count_outliers(df_in, col_name):
    _, _, _, _, minimum, maximum = valores_iqr(df_in, col_name)
    df_outliers = df_in.loc[(df_in[col_name] <= minimum) | (df_in[col_name] >= maximum)]
    return df_outliers.shape[0]

def replace_outlier(df_in, col_name):
    q1 = df_in[col_name].quantile(0.25)
    q3 = df_in[col_name].quantile(0.95)
    iqr = q3-q1 #Interquartile range
    fence_low  = q1-1.5*iqr
    fence_high = q3+1.5*iqr
    df_out = df_in.copy()
    outliers = ~df_out[col_name].between(fence_low, fence_high, inclusive=False)
    df_out.loc[outliers, col_name] = df_out.loc[~outliers, col_name].mean()

    return df_out

def outliers(data):
    
    """
    Esta funcion muestra la cantidad de los valores outliers, y reemplaza los outliers por la media de la columna deltainteger

    """

    for k, v in data.items():
        if data.shape[0] > 0:

            q1 = v.quantile(0.25)
            q3 = v.quantile(0.95) 
            irq = q3 - q1
            v_col = v[(v <= q1 - 1.5 * irq) | (v >= q3 + 1.5 * irq)]
            perc = np.shape(v_col)[0] * 100.0 / np.shape(data)[0] 
            print("Columna %s outliers = %.2f%%" % (k, perc))
        else:
            perc = 0 
            print("Columna %s outliers = %.2f%%" % (k, perc))
    if perc <= 5:
        data1 = replace_outlier(data, "deltainteger")

    else:
        data1 = data

    return data1

La funcion ***is_weekday*** obtiene de la columna de tiempo datetime, si es un dia de la semaan o un fin de semana.

In [32]:
def is_weekday(ds):
    date = pd.to_datetime(ds)
    return (date.weekday() < 5)


La funcion ***prophe*** realiza todo el el codigo para obtener un modelo basado en la libreria Prophet, ademas de la metrica. <br>
***prophe*** consta de varias funciones creadas anteriormente y nuevas transformaciones que son necesarias para el desarrollo del modelo predictivo.

* La libreria de prophet requiere dos entradas especiales con los nombres "y" , "ds". Es necesario renombrar las columna de deltainteger y SAMPLETIME. Ademas, se elimina la columna ID.
* Se añaden las columnas de la funcion ***is_weekday*** para detectar en funcion del tiempo si es fin de semana o no. 
* Algunos IDs no tienen datos suficientes por lo que es necesario identificarlos dentro del programa con una funcion if shape[0] >2.
* Se crea un datetime con las fechas especiales de los dias feriados de la comunidad valenciana, los cuales pueden influir en los picos de consumo

**1- Prediccion diaria**

* Para determinar la diferencia en dias desde la ultima fecha del dataset y la prediccion, se crea la variable delta2.
* Se prepara el Prophet añadiendo los holydays y las vacaciones de España.
* La funcion ***dimension*** se encarga de seleccionar los 7 dias antes de la ultima fecha del dataset (df1) para posteriormente ser añadido a la prediccion y de esta manera obtener el valor de RMSE, encontrando los nuevos valores predichos seran validado por los valores de test que corresponden a los 7 dias anteriores a la fecha del dataset.<br> En caso de que no hayan 7 datos, la funcion se encarga de añadir los valores que encuentre a la variable future, ya sean 6,5,4,3,2,1 datos de fechas para realizar la prediccion.  
* Se realiza la prediccion, añadiendo las columnas de la funcion is_weekday.
* Las predicciones con resultado negativo seran reemplazadas por los valores de cero, que me representar un consumo 0 de agua para ese contador en una fecha especifica. 
* Por ultimo se determina el valor RMSE usando el valor de test (ultimos 7 dias del dataset) y los 7 dias anteriores que predijo el algoritmo (predice 14 dias dependiendo del tamaño de df1). Esto esta sujeto a cambios, cuando no hay mas de 7 datos o 2 datos en df1, la metrica se calculara con los valores que hayan.

**2- Prediccion semanal**

* Se trabaja sobre el dataset llamado data_week el cual es definido antes de usar la funcion prophe.
* Usando la funcion ***group***, se agrupan los valores por semana
* La libreria de prophet requiere dos entradas especiales con los nombres "y" , "ds". Es necesario renombrar las columna nuevamente: deltainteger y SAMPLETIME.
* La variable delta3 es la cantidad de semanas que faltan a partir de la ultima fecha de data_week hasta la semana de prediccion requerida
* La funcion dimension2 se encarga de determinar si data_week tiene una dimension <2. Y de esta manera por la falta de informacion la prediccion y el valor RMSE sera igual a cero. 
* Si la funcion dimension2 encuentra un valor mayor a 2 en data_week realizara la prediccion con los datos utilizando las ultimas dos semanas de data_week mas las semanas de la prediccion requeridas (4 semanas para predecir).

**3- Score**
* La funcion metrica, obtiene el score final definido como Metrica = 0.5*(daily_RMSE) + 0.5*(week_RMSE)
* Se guardan los valores en un dataframe.

**4- Salida**
* La salida de la funcion ***prophe*** da como resultado la concatenacion de los valores predichos para los 7 dias de la semana mas dos valores que corresponden a dos semanas. 

In [33]:
def prophe(df, data_week):

    """
    Esta funcion predice los 10 valores pedidos por datathon, 7 dias + 2 semanas
    Ademas de las metricas de evaluacion AVERAGE RMSE
    """
    
    proph = True
    
    # i se encarga de guardar el ID que se obtiene al momento de la ejecucion del programa
    i = df['ID'].unique()
    # Se cambia el orden las columnas 
    df.reset_index(inplace=True)
    df.drop(df.columns[[1]], axis = 1, inplace=True) 
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    # Se renombran las columnas como "y" , "ds" para utilizar prophet
    df = df.rename(columns={"deltainteger":"y", "SAMPLETIME": "ds"}) 
    
    #Se eliminan los NaN en caso de que quedara alguno.
    df = df.dropna()

    # Añadimos la condicion de Marcar fin de semana
    df['weekday'] = df['ds'].apply(is_weekday)
    df['weekend'] = ~df['ds'].apply(is_weekday)
    
    # Se define una nueva variable df1
    df1 = df

    # Checkeamos si existen filas sin datos suficientes para realizar la prediccion:
    if df.shape[0] >= 2:
        proph = True
    else:
        print("AQUI NO HACE EL PROPHET PORQUE NO HAY DATOS")
        proph = False
        
    ## PRIMERO VAMOS A AGREGAR DENTRO DEL PROPHET LOS DIAS propios FERIADOS
    
    if proph:
        holidays = pd.DataFrame({
            'holiday': 'cvalencia',
            'ds': pd.to_datetime(['2019-01-01',
                                    '2019-03-19',
                                    '2019-04-19',
                                    '2019-04-22',
                                    '2019-05-01',
                                    '2019-06-24',
                                    '2019-08-15',
                                    '2019-10-09',
                                    '2019-10-12',
                                    '2019-11-01',
                                    '2019-12-06',
                                    '2019-12-25',
                                    '2020-01-01',
                                    '2020-02-06',
                                    '2020-02-14',
                                    ]),
            'lower_window': 0,
            'upper_window': 1,
        })

        ## SACAMOS LA PREDICCIÓN DEL DATATHON PRIMERO
        print("MODELO DATATHON.")            
        
        # Encontramos los dias que necesitamos predecir a partir de la ultima fecha que tiene el dataframe hasta el ultimo dia de la prediccion.
        
        timestamp = pd.Timestamp('2020-02-07')
        delta = timestamp - max(df1.ds) 
        delta2 = delta.days 
        
        # Preparamos el prophet
        m = Prophet(interval_width=0.95, daily_seasonality=True, holidays=holidays) 
        # Añadimos seasonalitys en función de si es finde o no
        m.add_seasonality(name='weekday', period=1, fourier_order=3, condition_name='weekday')
        m.add_seasonality(name='weekend', period=1, fourier_order=3, condition_name='weekend')
        # Añadimos todas las vacaciones de españa como holidays
        m.add_country_holidays(country_name='ES')
        m.fit(df1) 
        
        ### PREDICCIÓN PARA LOS SIGUIENTES 7 DÍAS
        future = m.make_future_dataframe(periods=delta2, freq="D")  
        # Nos quedamos con los primeros 7 dias. del 1 al 7.
        future = future.iloc[-7:] 
        future.reset_index(inplace = True, drop = True) 
        
        # Se crea una funcion dimension para primero: determinar si el tamaño de df1 es superior a 7 o menor. En caso de serlo, se seleccionaran los utimos 7 datos de fechas de df1... 
        # ...y se añaden al dataframe future con los 7 nuevos datos que son los 7 dias de la semana 01/02/2020 - 07/02/2020.  Se forma un nuevo dataframe de 14 filas en total de las fechas a predecir. 
        # En caso de que solo se encuentren menos de 7 valores, la funcion dimension se encargara de añadir las ultimas 6,5,4,3,2,o 1 fechas que existan para realizar la prediccion. 
        # Esto se hace con el fin de determinar el valor RMSE y asi obtener los valores test y pred del modelo. Prediciendo los valores de los 7 dias o menos dias anteriores a la fecha de la... 
        # ... prediccion real.
        
        def dimension(tt, future): 
            if int(tt.shape[0]) >=7:
                new_row = {'ds': tt["ds"].iloc[-7]} 
                new_row2 = {'ds': tt["ds"].iloc[-6]} 
                new_row3 = {'ds': tt["ds"].iloc[-5]} 
                new_row4 = {'ds': tt["ds"].iloc[-4]} 
                new_row5 = {'ds': tt["ds"].iloc[-3]} 
                new_row6 = {'ds': tt["ds"].iloc[-2]} 
                new_row7 = {'ds': tt["ds"].iloc[-1]} 

                future = future.append(new_row, ignore_index=True)
                future = future.append(new_row2, ignore_index=True)
                future = future.append(new_row3, ignore_index=True)
                future = future.append(new_row4, ignore_index=True)
                future = future.append(new_row5, ignore_index=True)
                future = future.append(new_row6, ignore_index=True)
                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)

            elif int(tt.shape[0]) ==6:

                new_row2 = {'ds': tt["ds"].iloc[-6]} 
                new_row3 = {'ds': tt["ds"].iloc[-5]} 
                new_row4 = {'ds': tt["ds"].iloc[-4]} 
                new_row5 = {'ds': tt["ds"].iloc[-3]} 
                new_row6 = {'ds': tt["ds"].iloc[-2]} 
                new_row7 = {'ds': tt["ds"].iloc[-1]} 

                future = future.append(new_row2, ignore_index=True)
                future = future.append(new_row3, ignore_index=True)
                future = future.append(new_row4, ignore_index=True)
                future = future.append(new_row5, ignore_index=True)
                future = future.append(new_row6, ignore_index=True)
                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)

            elif int(tt.shape[0]) ==5:

                new_row3 = {'ds': tt["ds"].iloc[-5]} 
                new_row4 = {'ds': tt["ds"].iloc[-4]} 
                new_row5 = {'ds': tt["ds"].iloc[-3]} 
                new_row6 = {'ds': tt["ds"].iloc[-2]} 
                new_row7 = {'ds': tt["ds"].iloc[-1]} 

                future = future.append(new_row3, ignore_index=True)
                future = future.append(new_row4, ignore_index=True)
                future = future.append(new_row5, ignore_index=True)
                future = future.append(new_row6, ignore_index=True)
                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)

            elif int(tt.shape[0]) ==4:


                new_row4 = {'ds': tt["ds"].iloc[-4]} 
                new_row5 = {'ds': tt["ds"].iloc[-3]} 
                new_row6 = {'ds': tt["ds"].iloc[-2]} 
                new_row7 = {'ds': tt["ds"].iloc[-1]} 


                future = future.append(new_row4, ignore_index=True)
                future = future.append(new_row5, ignore_index=True)
                future = future.append(new_row6, ignore_index=True)
                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)


            elif int(tt.shape[0]) ==3:


                new_row5 = {'ds': tt["ds"].iloc[-3]} 
                new_row6 = {'ds': tt["ds"].iloc[-2]} 
                new_row7 = {'ds': tt["ds"].iloc[-1]} 

                future = future.append(new_row5, ignore_index=True)
                future = future.append(new_row6, ignore_index=True)
                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)

            elif int(tt.shape[0]) ==2:

                new_row6 = {'ds': tt["ds"].iloc[-2]} 
                new_row7 = {'ds': tt["ds"].iloc[-1]} 

                future = future.append(new_row6, ignore_index=True)
                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)

            elif int(tt.shape[0]) ==1:

                new_row7 = {'ds': tt["ds"].iloc[-1]} 

                future = future.append(new_row7, ignore_index=True)
                future.sort_values(by = "ds", inplace=True, ascending=True) 
                future.reset_index(inplace = True, drop = True)
            return future
        # Aplicamos la funcion dimension 
        future = dimension(df1, future)  

        # Realizamos la prediccion y añadimos las columnas que me representan los dias o fines de semana
        future['weekday'] = future['ds'].apply(is_weekday)
        future['weekend'] = ~future['ds'].apply(is_weekday)
        forecast = m.predict(future)
        
        # Creamos una nueva variable para seleccionar unicamente los dos valores de interes que con la fecha "ds" y deltainteger "yhat".
        forecast1 = forecast.copy()    
        forecast1 = forecast[["ds","yhat"]] 
        # Las predicciones negativas seran reemplazadas por los valores de cero.
        forecast1["yhat"].mask(forecast1["yhat"] < 0, 0, inplace = True) 
        # Seleccionamos los ultimos 7 dias de interes 
        forecast_1 = forecast1.iloc[-7:] 
        forecast_1.reset_index(inplace = True, drop = True)
        # guardamos en una nueva variable los datos para posteriormente ser concatenados con la prediccion semanal.
        frames = [forecast_1[["ds","yhat"]]]   
        result = pd.concat(frames)
        
        
        # Metrica de evaluacion primer modelo - manual
        
        def rango(tt, forecast1):
            if int(tt.shape[0]) >= 7:
                y1 = sqrt(mean_squared_error(tt[['y']].iloc[-7], forecast1[['yhat']].iloc[-14])) 
                y2 = sqrt(mean_squared_error(tt[['y']].iloc[-6], forecast1[['yhat']].iloc[-13])) 
                y3 = sqrt(mean_squared_error(tt[['y']].iloc[-5], forecast1[['yhat']].iloc[-12])) 
                y4 = sqrt(mean_squared_error(tt[['y']].iloc[-4], forecast1[['yhat']].iloc[-11])) 
                y5 = sqrt(mean_squared_error(tt[['y']].iloc[-3], forecast1[['yhat']].iloc[-10])) 
                y6 = sqrt(mean_squared_error(tt[['y']].iloc[-2], forecast1[['yhat']].iloc[-9])) 
                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y1 + y2 + y3 + y4 + y5 + y6 + y7)/7 

            elif int(tt.shape[0]) == 6:
                y2 = sqrt(mean_squared_error(tt[['y']].iloc[-6], forecast1[['yhat']].iloc[-13])) 
                y3 = sqrt(mean_squared_error(tt[['y']].iloc[-5], forecast1[['yhat']].iloc[-12])) 
                y4 = sqrt(mean_squared_error(tt[['y']].iloc[-4], forecast1[['yhat']].iloc[-11])) 
                y5 = sqrt(mean_squared_error(tt[['y']].iloc[-3], forecast1[['yhat']].iloc[-10])) 
                y6 = sqrt(mean_squared_error(tt[['y']].iloc[-2], forecast1[['yhat']].iloc[-9])) 
                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y2 + y3 + y4 + y5 + y6 + y7)/6

            elif int(tt.shape[0]) == 5:

                y3 = sqrt(mean_squared_error(tt[['y']].iloc[-5], forecast1[['yhat']].iloc[-12])) 
                y4 = sqrt(mean_squared_error(tt[['y']].iloc[-4], forecast1[['yhat']].iloc[-11])) 
                y5 = sqrt(mean_squared_error(tt[['y']].iloc[-3], forecast1[['yhat']].iloc[-10])) 
                y6 = sqrt(mean_squared_error(tt[['y']].iloc[-2], forecast1[['yhat']].iloc[-9])) 
                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y3 + y4 + y5 + y6 + y7)/5

            elif int(tt.shape[0]) == 4:

                y4 = sqrt(mean_squared_error(tt[['y']].iloc[-4], forecast1[['yhat']].iloc[-11])) 
                y5 = sqrt(mean_squared_error(tt[['y']].iloc[-3], forecast1[['yhat']].iloc[-10])) 
                y6 = sqrt(mean_squared_error(tt[['y']].iloc[-2], forecast1[['yhat']].iloc[-9])) 
                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y4 + y5 + y6 + y7)/4

            elif int(tt.shape[0]) == 3:

                y5 = sqrt(mean_squared_error(tt[['y']].iloc[-3], forecast1[['yhat']].iloc[-10])) 
                y6 = sqrt(mean_squared_error(tt[['y']].iloc[-2], forecast1[['yhat']].iloc[-9])) 
                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y5 + y6 + y7)/3


            elif int(tt.shape[0]) == 2:

                y6 = sqrt(mean_squared_error(tt[['y']].iloc[-2], forecast1[['yhat']].iloc[-9])) 
                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y6 + y7)/2

            elif int(tt.shape[0]) == 1:

                y7 = sqrt(mean_squared_error(tt[['y']].iloc[-1], forecast1[['yhat']].iloc[-8])) 

                mean = (y7)/1

            return mean 
        

            # Obtenemos el valor de la metrica RMSE de los ultimos X dias del dataset.
        d = rango(df1, forecast1) 
                    
        
        ## SACAMOS LA PREDICCIÓN DE LAS SIGUIENTES DOS SEMANAS 
        # trabajamos sobre el dataset data_week
            
        
        # Organizamos el dataset por semanas
        data_week = group(data_week, "SAMPLETIME", "1W")
        data_week.reset_index(inplace=True, drop = False)
        data_week.drop(data_week.columns[[1]], axis = 1, inplace=True) 
        # Cambiamos el orden de las columnas para ingresar los valores al MODELO Prophet
        cols = data_week.columns.tolist()
        cols = cols[-1:] + cols[:-1]
        data_week = data_week[cols] 
        # Renombramos las variables deltainteger: "y", SAMPLETIME: "ds"
        data_week = data_week.rename(columns={"deltainteger":"y", "SAMPLETIME": "ds"})  
        
        # Encontramos los dias que necesitamos predecir a partir de la ultima fecha que tiene el dataframe hasta el ultimo dia de la prediccion, para posterirmente convertirlo en semanas.
        timestamp2 = pd.Timestamp('2020-02-14')
        delta3 = timestamp2 - max(data_week.ds)   
        delta3 = delta3.days 
        delta3= round(float((delta3%365)/7)) 
        
        #Segundo modelo
        
        # La funcion dimension2  determina el tamaño de data_week en caso de ser muy pequeño <2 la prediccion y el error RMSE sera 0, por la falta de datos. 
        def dimension2(tt2,delta3): 

            if int(tt2.shape[0]) >=2:
                m2 = Prophet(interval_width=0.95, daily_seasonality=True, holidays=holidays)    
                m2.add_country_holidays(country_name='ES')
                m2 = m2.fit(tt2)   
                future2 = m2.make_future_dataframe(periods=delta3, freq="W")   

                future2 = future2.iloc[-2:] 
                future2.iloc[-2] = future2.iloc[-2].apply(lambda dt: dt.replace(day=7, month = 2, year = 2020))
                future2.iloc[-1] = future2.iloc[-1].apply(lambda dt: dt.replace(day=14, month = 2, year = 2020)) 
                dw = data_week[-2:][['ds']]
                future2 = pd.concat([dw, future2]) 
                future2.reset_index(inplace=True, drop = True)
                forecast2 = m2.predict(future2)

                forecast3_ = forecast2[['ds', 'yhat']]
                forecast3_["yhat"].mask(forecast3_["yhat"] < 0, 0, inplace = True) 
                forecast_3_ = forecast3_.iloc[-2:]
                forecast_3_.reset_index(inplace = True, drop = True)
                forecast_3_ = forecast_3_

                # Error RMSE 
                y1 = sqrt(mean_squared_error(tt2[['y']].iloc[-2], forecast3_[['yhat']].iloc[-4]))
                y2 = sqrt(mean_squared_error(tt2[['y']].iloc[-1], forecast3_[['yhat']].iloc[-3]))
                week = (y1 + y2)/2 

            else: 
                diccion = {'ds':[0,0], 'yhat': [0,0]}
                forecast3_ = pd.DataFrame(diccion)
                forecast_3_ = pd.DataFrame(diccion)


                week = 0 

            return forecast3_, forecast_3_, week 
        
        forecast3, forecast_3, w = dimension2(data_week, delta3)  
            
            
        # Se determina la Metrica final - Score Final
        
        def metrica(daily, week):
            Metrica = 0.5*(daily) + 0.5*(week)
            return Metrica

        #Juntamos los modelos

        frames1 = [result, forecast_3[["yhat"]]]
        result1 = pd.concat(frames1)
        result1.reset_index(drop=True, inplace=True) 
        
        #creamos una columna llamada ID con su respectivo numero de contador y eliminadmos la variable "ds"
        result1 = result1.drop(labels = "ds", axis = 1)
        result1.reset_index(drop=True, inplace=True) 
        result1['ID'] = ID
        result1.set_index('ID', inplace=True) 
        
        # Metricas            
        #creamos una variable para guardar el score final
        metri = metrica(d,w) 
        metricas2 = pd.DataFrame(np.array([metri]))
        metricas2 = pd.concat([metricas2]) 
        metricas2 = metricas2.rename(columns={0:"metrica"})
        metricas2.reset_index(drop = True, inplace = True)
        
        
        return result1, metricas2, proph 
        
    else: 
        return 0,0, proph  

La funcion ***resul*** crea un dataframe a partir de un diccionario, y se obtiene como salida los valores ID,Predicciones o ID, Score.

In [34]:
def resul(dict, ids_list):

    resultado = pd.DataFrame.from_dict(dict,orient='index')
    return  resultado

# Programa

In [35]:
final = pd.DataFrame()
final2 = pd.DataFrame()
ids_list = []
dict_result = {}
dict_metricas = {}

arr_random_ids = list(range(0,2747)) 

for ID in arr_random_ids:
    print(ID)

    data1= data.copy()

    hour(data1, "SAMPLETIME") 
    

    data1 = id(data1, ID, "SAMPLETIME")  
    
    negativo(data1)


    data1 = deltainteger(data1)
    
    # Se crea una condicion en caso de que el tamaño del data1 sea igual a cero. Se añaden las fechas faltantes con un valor de cero para que el programa funcione. 
    if int(data1.shape[0]) == 0:
        dt = pd.date_range("2019-02-01", "2020-01-31")
        dt = pd.DataFrame(dt) 
        dt.rename(columns={0: "SAMPLETIME"}, inplace=True) 
        dt["deltainteger"] = 0
        dt["ID"] = ID
        data1 = dt
    else:
        data1 = data1
        
    # Se creauna copia para utilizar data_week en la prediccion semanal
    data_week = data1.copy()  

    #Se agrupa por dia, ID el dataframe

    data1 = group(data1, "SAMPLETIME", "1D")
    
    
    # funcion para detectar la cantidad de outliers y reemplazar los valores por fuera del cuartil 0.95 para ser reemplazados por la media
    
    data1 = outliers(data1) 
    
    # Creamos una copia de data1    
    data2 = data1.copy()
    
    #Implemetamos el modelo Prophet con la funcion creada anteriormente prophe
    result, metricas2, proph = prophe(data2, data_week)
    
    # Creamos un dicionario con los valores de la prediccion y las metricas
    if proph:
        
        dict_result[ID] = result['yhat'].to_numpy() 
        dict_metricas[ID] = metricas2['metrica'].to_numpy()
        # Guardamos los valores de los ID
        ids_list.append(ID)

# GUardamos las variables de salida para las predicciones y las metricas
resul(dict_result, ids_list).to_csv('C:/Users/pulzara/Documents/Datathon/predicciones/F.txt', sep = '|', header = False)
resul(dict_metricas, ids_list).to_csv('C:/Users/pulzara/Documents/Datathon/predicciones/M.txt', sep = '|', header = False)

0
Columna ID outliers = 100.00%
Columna deltainteger outliers = 0.00%
MODELO DATATHON.
1
Columna ID outliers = 100.00%
Columna deltainteger outliers = 2.47%
MODELO DATATHON.
